# This is the code which solves the Static Traffic Assignment Problem

We want to solve the following problem (TAP-C):
\begin{align}
\min_{f, h} &\sum_{a} \int_{0}^{f_a} c_a(s)\; \text{d}s
\\
\text{s.t.  } & \;\; f = \Delta h
\\
& \;\; 
h \geq 0
\\
& \;\; 
A h = d
\\
& \;\; 
f \leq u
\end{align}

This problem is hard to compute, even if it is a convex problem. Computing all the path of a network is NP-hard (with respect to the number of edges and links).

Nevertheless, the TAP can be solve using a Frank-Wolf algorithm:
\begin{align}
&\text{1. TO DO }
\\
&\text{2. }
\end{align}


#### Remarks
First, we tried to solve the STA with CVX.
This does not work because the values of the travel time are to big. 

Secondly, we solve the STA using links flow. 
This does not help us in our project because we need the path flow to compute the dual of the TAP-C.

Thirdly, we solve the STA using paths flow. 
The issue here is that finding every path in a network in a NP-complete problem. 
But if we can compute every path (we only need to do it one time), then it is really fast to compute the shortest path. We do not need to do a Dikjstra's algorithm to compute the shortest path, only a big matrix multiplication and a array sorting are enough. This might be more difficult on a laptop. But it might be faster on a HPC.

Finally, we solve the STA using links flow and we compute the paths during the all or nothing step of the Frank Wolf algorithm. TO BE DONE.

In [1]:
import numpy as np
import scipy.sparse

In [30]:
I210 = 'data/I210'
Chic = 'data/Chicago'
Anah = 'data/Anaheim'
Siou = 'data/SiouxFalls'
Brae = 'data/braess'

network = Brae
debug = True

## 1. We load the graph and the demand
Both graph and demand are in csv file, we load them

In [31]:
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)

### We clean the data

In [49]:
"""
We need to demand to be an array like:
[
[o1, d1, demand from o1 to d1],
...,
[on, dn, demand from on to dn]
]

We need the graph to be an array like:
[
[id link 1, node origin link 1, node destination link 1, a0, a1, a2, a3, a4],
...,
[id link n, node origin link n, node destination link n, a0, a1, a2, a3, a4]
]
where the node are indexed from 0 to nb_nodes-1,
    the links are indexed from 0 to nb_links-1,
    and a0, ..., a4 are the coeficient to calculate the travel time of one link as a function
    of the flow on the link: t = a0 + a1 * f + a2 * f**2 + a3 * f**3 + a4 * f**4

One can add some checks here to be sure that demand and graph respect this format!

Some demand and graph data can be found on:
    https://github.com/bstabler/TransportationNetworks
    Here one need to perprocess the data before using this code
"""

def cleaning_input(graph, demand):
    # in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix (2d array)
    try:
        demand.shape[1]
    except:
        demand = np.array([demand])
    nb_ods = int(demand.shape[0])

    # in the case where the index of the od pairs does not begin by 0, we rename the od pairs
    first_index_od = min(np.min(graph[:,1]), np.min(graph[:,2]))
    graph[:,1] = graph[:,1]-first_index_od
    graph[:,2] = graph[:,2]-first_index_od
    demand[:,0] = demand[:,0] - first_index_od
    demand[:,1] = demand[:,1] - first_index_od
    return graph, demand

graph, demand = cleaning_input(graph, demand)
demand[0][2] = 10
nb_links = int(np.max(graph[:,0])+1)
nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
nb_ods = int(demand.shape[0])
if debug:
    print("nb nodes = " + str(nb_nodes))
    print("nb links = " + str(nb_links))
    print("nb ods = " + str(nb_ods))

nb nodes = 4
nb links = 5
nb ods = 1


Then, we define the function which gives the travel time as a function of the flow

In [50]:
## Here Max float is fixed to not overlap the maximum float number when we compute the potential function
max_float = 1e+10
if debug:
    print(max_float)

"""
We define the travel time of each link as a function of the flow on each link.
This function is given by the network topology (in the graph file) and by the capacity of each link.
One row of the graph is:
[id link 1, node origin link 1, node destination link 1, a0, a1, a2, a3, a4]
Each row correspond to one link.
The travel time of the link is t = a0 + a1 * f + a2 * f**2 + a3 * f**3 + a4 * f**4 if f < c 
    It is t = + inf (max_float) is f >= c
"""
def travel_time(graph, f, c = -1):
    if c == -1:
        c = [max_float for i in range(len(f))]
    # here we need to have the same indexation for graph and f. That why we need to store graph in a dictionnary
    tt_tmp = graph[:,3] + graph[:,4]*f + graph[:,5]*(f**2) + graph[:,6]*(f**3) + graph[:,7]*(f**4)
    tt_tmp = [tt_tmp[i] if f[i]<c[i] else max_float for i in range(len(f))]
    return tt_tmp

10000000000.0


## 2. We compute the all or nothing flow allocation

To use the Dijkstra's algorithm class of scipy we need to define the adjacent matrix of the graph

In [51]:
"""
We store the travel time of each link in a sparce adjacency matrix G.
Given a flow allocation f and a capacities constraints c, we compute 
the travel time of each link and store it in G.

This allow us after to us the Disjtra's algorithm of the class scipy.sparse.csgraph
"""

def update_travel_time_from_tt(graph, tt):
    G = np.zeros(shape=(nb_nodes,nb_nodes))
    G = scipy.sparse.lil_matrix(G)
    for i in range(graph.shape[0]):
        G[int(graph[i][1]),int(graph[i][2])] = tt[i]
    G = G.tocsr()
    return G

def update_travel_time_from_flow(graph, f, c=-1):
    return update_travel_time_from_tt(graph, travel_time(graph, f, c))

In [52]:
G = update_travel_time_from_flow(graph, np.zeros(nb_links)) # , [1,1,-1,1,1]))
if debug:
    print(G)

  (0, 1)	1.0
  (0, 2)	2.0
  (1, 2)	0.25
  (1, 3)	2.0
  (2, 3)	1.0


In [53]:
from scipy.sparse.csgraph import dijkstra

In [54]:
"""
Because the classic Frank Wolf algorithm does not take into account the paths,
we need to store the path in memory. We do it by using Object and a dictionnary 
called paths_used.

This idea is that we do not want to compute all the possible paths of the network
before the beginning of the algorithm. So we compute the paths used during the algorithm,
and we store them in memory.

We build at the same time the incidence matrix.

CAREFUL. We use the number of links of the graph in the calcul of the hash of every path.
This variable should be a parameter.
"""

class path:
    def __init__(self,links):
        self.links = links
        self.__flow = 0
    def set_flow(self, flow):
        self.__flow = flow
    def get_flow(self,):
        return self.__flow
    def __eq__(self, other):
        return np.all(self.links == other.links)
    # I am not sure about the hash table structure in Python. I am used to Java.
    def __hash__(self):
        return hash(np.sum([hash(self.links[i]*(nb_links**i)) for i in range(len(self.links))]))
    def __str__(self):
        return str(self.__flow) + " is on " + str(self.links) 

In [55]:
if debug:
    paths_used = {}
    print(paths_used)
    p = path([0, 1, 2, 3])
    paths_used[hash(p)] = (p, 0)
    print(paths_used)
    p = path([0, 1, 1, 3])
    paths_used[hash(p)] = (p, 0)
    print(paths_used)
    p = path([0, 0, 2, 3])
    paths_used[hash(p)] = (p, 0)
    print(paths_used)

{}
{430: (<__main__.path object at 0x11061b9b0>, 0)}
{430: (<__main__.path object at 0x11061b9b0>, 0), 405: (<__main__.path object at 0x11061bef0>, 0)}
{430: (<__main__.path object at 0x11061b9b0>, 0), 405: (<__main__.path object at 0x11061bef0>, 0), 425: (<__main__.path object at 0x11061b240>, 0)}


In [56]:
print(demand[:,0])
print(demand[:,1])

[0.]
[3.]


Now let's compute the all or nothing allocation

In [57]:
"""
This cell is the main point of the entire solver.
It is the all or nothing allocation.

"""

# graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
graph_dict = {}
for i in range(graph.shape[0]):
    try: 
        graph_dict[int(graph[i][1])]
    except:
        graph_dict[int(graph[i][1])] = {}
    graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])
    
# computing the all or nothing flow
paths_used = {}

# the all or nothing allocation
# TO DO, PASS paths_used AS A PARAMETER OF THE FUNCTION
def all_or_nothing():
    # k is the next index to give to a new path.
    global k
    # paths_used_tmp is the set of the path in the all or nothing allocation of this step
    paths_used_tmp = {}
    # faon is the flow allocation of the all or nothing allocation
    faon = np.zeros(nb_links)
    
    # using scipy to compute dijkstra
    """
    HERE ONE CAN CHANGE THE ALGORITHM TO BE FASTER. IN THIS CASE WE DO NOT TAKE INTO ACCOUNT
    THE SMALL NUMBER OF O-D PAIRS.
    TO DO, ADD THE INDICE OF THE O-D PAIRS TO CONSIDER: 
    
    indices : array_like or int, optional
        if specified, only compute the paths for the points at the given indices.
    """
    dist_matrix, return_predecessors = dijkstra(G, return_predecessors = True)
    
    # for every origin destination pairs (i.e. one line of the demand file)
    for i in range(nb_ods):
        o_tmp = int(demand[i][0])
        d_tmp = int(demand[i][1])
        flow_tmp = demand[i][2]

        node_tmp = d_tmp
        links = []
        # using the dijkstra, we build the fastest path and we put the flow on it.
        while node_tmp != o_tmp:
            node_tmp_d = return_predecessors[o_tmp][node_tmp]
            # Here we need the graph_dict to recover the link id from the nodes id.
            link_tmp = graph_dict[node_tmp_d][node_tmp]
            # we recover the path from the predecessor of the 
            links.insert(0, link_tmp)
            faon[link_tmp] += flow_tmp
            node_tmp = node_tmp_d
        
        p = path(links)
        p.set_flow(flow_tmp)
        
        
        if debug:
            print(p)
        # If we do not already know p, we add it to the delta matrix (here a dict)
        if not hash(p) in paths_used:
            if debug:
                print(k)
            # we add p and his index
            paths_used[hash(p)] = (p, k)
            paths_used_tmp[hash(p)] = (p, k)
            k = k+1
        else:
            # we recover the index of p using the paths_used dict
            paths_used_tmp[hash(p)] = (p, paths_used[hash(p)][1])
    return faon, paths_used_tmp

We define the line search

In [58]:
if debug:
    k = 0 
    all_or_nothing()
    print(paths_used)

10.0 is on [0, 2, 4]
0
{110: (<__main__.path object at 0x1106163c8>, 0)}


In [59]:
"""
The function potential is used to compute the line search between 
the all or nothing allocation and the current flow allocation
The function potential returns the objective function corresponding to
the flow allocation f.

The function line search does a 1D line search.
"""

def potential(graph, f, c=-1):
    # this routine is useful for doing a line search
    # computes the potential at flow assignment f
    if c == -1:
        c = [max_float for i in range(len(f))] # this might be to much, to do once we have everything done
    # here we need to have the same indexation for graph and f.
    pot_tmp = graph[:,3]*f + 1/2*graph[:,4]*(f**2) + 1/3*graph[:,5]*(f**3) + 1/4*graph[:,6]*(f**4) + 1/5*graph[:,7]*(f**5)
    pot_tmp = [pot_tmp[i] if f[i]<c[i] else f[i]*max_float for i in range(len(f))]
    return np.sum(pot_tmp)


def line_search(f, res=20):
    debug_local = False
    # on a grid of 2^res points bw 0 and 1, find global minimum
    # of continuous convex function
    # here we do a bisection
    d = 1. / (2**res - 1)
    l, r = 0, 2**res - 1
    while r - l > 1:
        if f(l * d) <= f(l * d + d):
            return l * d
        if f(r * d - d) >= f(r * d):
            return r * d
        # otherwise f(l) > f(l+d) and f(r-d) < f(r)
        m1, m2 = (l + r) / 2, 1 + (l + r) / 2
        if debug_local:
            print(l * d, end=": ")
            print(f(l * d))
            print(r * d, end=": ")
            print(f(r * d))
            print(str(m1 * d) + " = " + str(f(m1 * d)))
            print(str(m2 * d) + " = " + str(f(m2 * d)))
            print()
        if f(m1 * d) < f(m2 * d):
            r = m1
        if f(m1 * d) > f(m2 * d):
            l = m2
        if f(m1 * d) == f(m2 * d):
            return m1 * d
    return l * d

Now let run the Frank-Wolf's algorithm with a line search to find alpha

In [60]:
eps=1e-8
k = 0
paths_used = {}
f = np.zeros(nb_links)
G = update_travel_time_from_flow(graph, f)

f, paths_aux = all_or_nothing()
print(f)
G = update_travel_time_from_flow(graph, f)
print(paths_used)
path_matrix = np.zeros(len(paths_aux))

for val in paths_aux.values():
    print(val[1])
    path_matrix[val[1]] = val[0].get_flow()

print(path_matrix)

for i in range(1000):
    if i % 100 == 0:
        print(i)
    faon, paths_tmp = all_or_nothing() 
    s = line_search(lambda a: potential(graph, (1. - a) * f + a * faon))
    print("s: " + str(s))
    if s < eps:
        break
    path_matrix = np.append(path_matrix, np.zeros(len(paths_used)-path_matrix.shape[0]))
    path_matrix = (1-s) * path_matrix
    for val in paths_tmp.values():
        path_matrix[val[1]] += s * val[0].get_flow()
    f = (1. - s) * f + s * faon
    
    # also update the path flow
    ## TO DO ##
    G = update_travel_time_from_flow(graph, f)
print(path_matrix)

10.0 is on [0, 2, 4]
0
[10.  0. 10.  0. 10.]
{110: (<__main__.path object at 0x1106163c8>, 0)}
0
[10.]
0
10.0 is on [0, 3]
1
s: 0.25
10.0 is on [1, 4]
2
s: 0.23529399885195756
10.0 is on [0, 3]
s: 0.06217553210787696
10.0 is on [1, 4]
s: 0.031249046210177656
10.0 is on [0, 3]
s: 0.01715394482308952
10.0 is on [1, 4]
s: 0.009800593138303839
10.0 is on [0, 3]
s: 0.00571951223841394
10.0 is on [1, 4]
s: 0.0033751800684811454
10.0 is on [0, 3]
s: 0.0020056720851240354
10.0 is on [1, 4]
s: 0.0011971555680325735
10.0 is on [0, 3]
s: 0.0007153823991146187
10.0 is on [1, 4]
s: 0.0004289150253953282
10.0 is on [0, 3]
s: 0.00025697052610951357
10.0 is on [1, 4]
s: 0.00015425682227032818
10.0 is on [0, 3]
s: 9.298324721385273e-05
10.0 is on [1, 4]
s: 5.5074693364078183e-05
10.0 is on [0, 3]
s: 3.147125335090182e-05
10.0 is on [1, 4]
s: 1.9788742747551364e-05
10.0 is on [0, 3]
s: 1.2874604635727729e-05
10.0 is on [1, 4]
s: 7.62939453125e-06
10.0 is on [0, 3]
s: 4.768372491526819e-06
10.0 is on [1,

In [64]:
for p in paths_used.values():
    index = p[1]
    p[0].set_flow(path_matrix[index])
    print(str(p[0]))

print(demand)
print(G)
print(f)



5.000008529100963 is on [0, 2, 4]
2.49999853134832 is on [0, 3]
2.499992939550717 is on [1, 4]
[[ 0.  3. 10.]]
  (0, 1)	1.7500007060449283
  (0, 2)	2.0
  (1, 2)	0.25
  (1, 3)	2.0
  (2, 3)	1.750000146865168
[7.50000706 2.49999294 5.00000853 2.49999853 7.50000147]


In [62]:
print("[", end="")
for i in range(len(travel_time(graph, f))):
    print(travel_time(graph, f)[i], end=", ")
print()

[1.7500007060449283, 2.0, 0.25, 2.0, 1.750000146865168, 


In [63]:
print(graph)
print(paths_used)

[[0.   0.   1.   1.   0.1  0.   0.   0.  ]
 [1.   0.   2.   2.   0.   0.   0.   0.  ]
 [2.   1.   2.   0.25 0.   0.   0.   0.  ]
 [3.   1.   3.   2.   0.   0.   0.   0.  ]
 [4.   2.   3.   1.   0.1  0.   0.   0.  ]]
{110: (<__main__.path object at 0x1106163c8>, 0), 15: (<__main__.path object at 0x1106e64a8>, 1), 21: (<__main__.path object at 0x1106e62e8>, 2)}


In [ ]:
TO DO: ADD THE CORRECT OUTPUT